In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision.models.detection import ssd300_vgg16
from torchvision.datasets import CocoDetection
from pycocotools.cocoeval import COCOeval
import json
import os

model_path = "full_model_epoch_75.pth"


model = ssd300_vgg16(pretrained=False)
num_classes = 2
model.head.classification_head.num_classes = num_classes


model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
model.eval()


transform = transforms.Compose([transforms.ToTensor()])


def run_inference(image, model, transform):
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image_tensor)
    return outputs


class PKLotCOCODataset(CocoDetection):
    def __init__(self, root, annFile, transform=None):
        super(PKLotCOCODataset, self).__init__(root, annFile)
        self.transform = transform

    def __getitem__(self, idx):
        img, target = super(PKLotCOCODataset, self).__getitem__(idx)
        if self.transform is not None:
            img = self.transform(img)
        return img, target


test_dataset = PKLotCOCODataset(
    root="../data2/test",
    annFile="../data2/test/_annotations.coco.json",
    transform=transform,
)


def compute_metrics(model, dataset):
    coco_gt = dataset.coco
    coco_results = []

    for idx in range(len(dataset)):
        img, target = dataset[idx]

        if len(target) == 0:
            continue

        image_id = target[0].get("image_id", idx)
        original_image = dataset._load_image(image_id)

        outputs = run_inference(original_image, model, transform)

        for i in range(len(outputs[0]["boxes"])):
            box = outputs[0]["boxes"][i].cpu().numpy().tolist()
            score = outputs[0]["scores"][i].cpu().item()
            label = outputs[0]["labels"][i].cpu().item()

            if score > 0.5:
                x_min, y_min, x_max, y_max = box
                bbox = [x_min, y_min, x_max - x_min, y_max - y_min]
                coco_results.append(
                    {
                        "image_id": int(image_id),
                        "category_id": int(label),
                        "bbox": bbox,
                        "score": score,
                    }
                )

    result_file_path = "coco_results.json"
    with open(result_file_path, "w") as f:
        json.dump(coco_results, f, indent=4)

    coco_dt = coco_gt.loadRes(result_file_path)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType="bbox")

    coco_eval.params.imgIds = [img_id for img_id in coco_gt.getImgIds()]

    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    precision = coco_eval.stats[0]
    recall = coco_eval.stats[1]
    mAP = coco_eval.stats[2]

    return precision, recall, mAP


precision, recall, mAP = compute_metrics(model, test_dataset)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"mAP: {mAP}")

In [3]:

result_file_path = 'coco_results.json'
coco_gt = test_dataset.coco  
coco_dt = coco_gt.loadRes(result_file_path)
coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')


coco_eval.params.imgIds = [img_id for img_id in coco_gt.getImgIds()]

coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()


precision = coco_eval.stats[0]
recall = coco_eval.stats[1]
mAP = coco_eval.stats[2]

Loading and preparing results...
DONE (t=0.79s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=68.65s).
Accumulating evaluation results...
DONE (t=1.18s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.904
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.984
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.979
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.834
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.931
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.984
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.028
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.231
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.931
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1